In [1]:
import torch
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Kaggle ortamında veri kümesinin yolu
data_path = "Çiçek veri kümesinin bulunduğu dizin (kaggle veri seti) "  # Çiçek veri kümesinin bulunduğu dizin

# Görseller için veri önişleme transformasyonu
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Görselleri 128x128 boyutuna yeniden boyutlandır
    transforms.ToTensor(),         # Görselleri tensör formatına çevir
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # RGB kanalları için normalize et
])

# Veri kümesini yükle (Klasör yapısına göre)
dataset = datasets.ImageFolder(root=data_path, transform=transform)  # Klasör yapısına göre veri kümesini yükle

# Veri kümesini eğitim ve test seti olarak ayır
train_size = int(0.8 * len(dataset))  # %80 eğitim için ayrılır
test_size = len(dataset) - train_size  # %20 test için ayrılır
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])  # Eğitim ve test setine böl

# Veri yükleyiciler (DataLoader) tanımla
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)  # Eğitim veri yükleyici, karıştırma açık
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)   # Test veri yükleyici, karıştırma kapalı

# Toplam veri sayısını yazdır
print(f"Toplam Görsel Sayısı: {len(dataset)}")  # Veri kümesindeki toplam görüntü sayısını yazdır
print(f"Eğitim Görselleri: {len(train_dataset)}, Test Görselleri: {len(test_dataset)}")  # Eğitim ve test setlerinin boyutlarını yazdır

SyntaxError: invalid syntax (2131674457.py, line 14)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# CNN Modeli Tanımlama
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)  # 3 giriş kanalı (RGB), 32 filtre
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # Boyut küçültme işlemi (2x2 alanları havuzlama)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)  # 32 giriş, 64 filtre (ikinci katman)
        self.fc1 = nn.Linear(64 * 32 * 32, 128)  # Fully connected katman, 64 filtre ve 32x32 boyut giriş
        self.fc2 = nn.Linear(128, 5)  # Çıkış katmanı, 5 sınıf (çiçek türleri)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))  # İlk konvolüsyon ve pooling işlemi
        x = self.pool(torch.relu(self.conv2(x)))  # İkinci konvolüsyon ve pooling işlemi
        x = x.view(-1, 64 * 32 * 32)  # Düzleştirme (Flatten)
        x = torch.relu(self.fc1(x))  # Fully connected katman
        x = self.fc2(x)  # Çıkış katmanı (sınıf tahmini)
        return x

# Model, Kayıp Fonksiyonu ve Optimizasyon Tanımı
model_cnn = CNN()  # CNN modelini başlat
criterion = nn.CrossEntropyLoss()  # Çok sınıflı sınıflandırma için uygun kayıp fonksiyonu
optimizer = optim.Adam(model_cnn.parameters(), lr=0.001)  # Adam optimizasyon algoritması, öğrenme oranı 0.001

# Eğitim ve Doğruluk/Kayıp Takibi
def train_cnn(model, train_loader, test_loader, criterion, optimizer, epochs=5):
    train_losses = []  # Eğitim kayıplarını izlemek için
    test_accuracies = []  # Test doğruluklarını izlemek için

    for epoch in range(epochs):
        model.train()  # Modeli eğitim moduna al
        running_loss = 0.0  # Epoch başına toplam kayıp

        for images, labels in train_loader:
            optimizer.zero_grad()  # Optimizasyonu sıfırla
            outputs = model(images)  # Modelden tahmin al
            loss = criterion(outputs, labels)  # Kayıp hesapla
            loss.backward()  # Geri yayılım (backpropagation)
            optimizer.step()  # Optimizasyon adımı
            running_loss += loss.item()  # Toplam kaybı artır

        train_losses.append(running_loss / len(train_loader))  # Ortalama eğitim kaybını kaydet
        test_accuracy = evaluate_cnn(model, test_loader)  # Test doğruluğunu hesapla
        test_accuracies.append(test_accuracy)  # Doğruluğu kaydet

        # Her epoch sonunda kayıp ve doğruluk yazdır
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {test_accuracy:.2f}%")

    return train_losses, test_accuracies  # Eğitim kaybı ve test doğruluklarını döndür

# Test Veri Kümesinde Değerlendirme
def evaluate_cnn(model, test_loader):
    model.eval()  # Modeli değerlendirme moduna al
    correct = 0  # Doğru tahmin sayısı
    total = 0  # Toplam veri sayısı

    with torch.no_grad():  # Tahmin sırasında gradyan hesaplamalarını kapat
        for images, labels in test_loader:
            outputs = model(images)  # Modelden tahmin al
            _, predicted = torch.max(outputs, 1)  # En yüksek olasılıklı sınıfı seç
            total += labels.size(0)  # Toplam veri sayısını artır
            correct += (predicted == labels).sum().item()  # Doğru tahmin sayısını artır

    return 100 * correct / total  # Doğruluk oranını yüzdelik olarak döndür

# Eğitim ve Grafikler
train_losses, test_accuracies = train_cnn(model_cnn, train_loader, test_loader, criterion, optimizer, epochs=5)  # CNN modelini eğit

# Grafik Çizdirme
plt.figure(figsize=(12, 5))  # Grafik boyutunu ayarla
plt.subplot(1, 2, 1)  # 1. grafik: Eğitim kayıpları
plt.plot(train_losses, label="Training Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.title("Training Loss Over Epochs")  

plt.subplot(1, 2, 2)  # 2. grafik: Test doğruluğu
plt.plot(test_accuracies, label="Test Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy (%)")
plt.legend()
plt.title("Test Accuracy Over Epochs")  
plt.show()  # Grafikleri göster

In [ ]:
from torchvision import models
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# ResNet-18 Transfer Learning
def transfer_learning_resnet(train_loader, test_loader, num_classes=5, epochs=5):
    from torchvision.models import ResNet18_Weights

    # Önceden eğitilmiş ResNet-18 modelini yükleme
    model_resnet = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)  # ResNet-18 modelini ImageNet ağırlıklarıyla yükle

    # Son katmanı (fully connected) değiştirme
    model_resnet.fc = nn.Linear(model_resnet.fc.in_features, num_classes)  # Çıkış sınıf sayısını çiçek sınıflarına uyacak şekilde ayarla

    # Kayıp fonksiyonu ve optimizer tanımlama
    criterion = nn.CrossEntropyLoss()  # Çok sınıflı sınıflandırma için uygun kayıp fonksiyonu
    optimizer = optim.Adam(model_resnet.parameters(), lr=0.001)  # Adam optimizasyon algoritması, öğrenme oranı 0.001

    # Eğitim
    train_losses = []  # Eğitim kayıplarını izlemek için liste
    for epoch in range(epochs):
        model_resnet.train()  # Modeli eğitim moduna al
        running_loss = 0.0  # Her epoch başına kayıp

        for images, labels in train_loader:
            optimizer.zero_grad()  # Optimizasyon gradyanlarını sıfırla
            outputs = model_resnet(images)  # Modelden tahmin al
            loss = criterion(outputs, labels)  # Kayıp hesapla
            loss.backward()  # Geri yayılım işlemini başlat
            optimizer.step()  # Optimizasyon adımı uygula
            running_loss += loss.item()  # Toplam kaybı güncelle

        train_losses.append(running_loss / len(train_loader))  # Ortalama epoch kaybını kaydet

        # Test doğruluğu hesaplama
        accuracy = evaluate_accuracy(model_resnet, test_loader)  # Test veri kümesinde doğruluğu hesapla
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {accuracy:.2f}%")  # Epoch sonuçlarını yazdır

    return model_resnet, train_losses  # Eğitimdeki kayıpları ve modeli döndür

# Test veri kümesinde doğruluk hesaplama
def evaluate_accuracy(model, loader):
    model.eval()  # Modeli değerlendirme moduna al
    correct = 0  # Doğru tahmin sayısı
    total = 0  # Toplam veri sayısı

    with torch.no_grad():  # Gradyan hesaplamalarını devre dışı bırak
        for images, labels in loader:
            outputs = model(images)  # Modelden tahmin al
            _, predicted = torch.max(outputs, 1)  # En yüksek olasılıklı sınıfı seç
            total += labels.size(0)  # Toplam veri sayısını güncelle
            correct += (predicted == labels).sum().item()  # Doğru tahminleri ekle

    return 100 * correct / total  # Yüzde olarak doğruluğu döndür

# Eğitim işlemi
print("=== ResNet-18 Transfer Learning ===")
model_resnet, train_losses_resnet = transfer_learning_resnet(train_loader, test_loader, epochs=5)  # ResNet-18 modelini eğit

# Grafik Çizdirme
plt.figure(figsize=(6, 5))  # Grafik boyutunu ayarla
plt.plot(train_losses_resnet, label="Training Loss (ResNet-18)")  # Eğitim kayıplarını çiz
plt.xlabel("Epochs")  # X ekseni başlığı
plt.ylabel("Loss")  # Y ekseni başlığı
plt.legend()  # Efsane ekle
plt.title("ResNet-18 Training Loss Over Epochs")  # Grafik başlığı
plt.show()  # Grafiği göster

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Model Performansını Değerlendirme Fonksiyonu
def evaluate_with_metrics(model, loader, class_names):
    model.eval()  # Modeli değerlendirme moduna al
    y_true = []  # Gerçek etiketler için liste
    y_pred = []  # Tahmin edilen etiketler için liste

    with torch.no_grad():  # Tahmin sırasında gradyan hesaplamayı kapat
        for images, labels in loader:
            outputs = model(images)  # Modelden tahmin al
            _, predicted = torch.max(outputs, 1)  # En yüksek olasılıklı sınıfı seç
            y_true.extend(labels.cpu().numpy())  # Gerçek etiketleri listeye ekle
            y_pred.extend(predicted.cpu().numpy())  # Tahmin edilen etiketleri listeye ekle
    
    # Metrikleri yazdırma
    print("Classification Report:")  # Sınıflandırma raporunu yazdır
    print(classification_report(y_true, y_pred, target_names=class_names))  # Sınıf bazlı metrikleri yazdır
    accuracy = accuracy_score(y_true, y_pred)  # Genel doğruluk oranını hesapla
    print(f"Accuracy: {accuracy:.2f}")  # Doğruluğu yazdır
    return accuracy  # Doğruluk oranını döndür

# Çiçek sınıflarını tanımlayın
class_names = ["Daisy", "Dandelion", "Rose", "Sunflower", "Tulip"]  # Çiçek sınıf isimleri

# CNN Model Performansı
print("=== CNN Model Performance ===")
cnn_accuracy = evaluate_with_metrics(model_cnn, test_loader, class_names)  # CNN modelini değerlendir

# ResNet-18 Model Performansı
print("\n=== ResNet-18 Model Performance ===")
resnet_accuracy = evaluate_with_metrics(model_resnet, test_loader, class_names)  # ResNet-18 modelini değerlendir

In [ ]:
from PIL import Image
from torchvision import transforms
import torch

# Çiçek sınıfları
class_names = ["Daisy", "Dandelion", "Rose", "Sunflower", "Tulip"]  # Tahmin edilecek çiçek sınıfları

# Görüntüyü tahmin etme fonksiyonu
def predict_image(model, image_path, class_names):
    # Modeli değerlendirme moduna al
    model.eval()  # Modeli tahmin moduna geçir (dropout gibi işlemler devre dışı)

    # Görüntü yükleme ve ön işleme
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # ResNet'in beklediği giriş boyutu
        transforms.ToTensor(),  # Görüntüyü tensöre dönüştür
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet için kullanılan normalizasyon
    ])
    
    # Görüntüyü aç ve ön işle
    image = Image.open(image_path).convert("RGB")  # Görüntüyü RGB formatına çevir
    image_tensor = transform(image).unsqueeze(0)  # Tensöre dönüştür ve batch boyutu ekle (unsqueeze)

    # Tahmin yap
    with torch.no_grad():  # Gradyan hesaplamayı devre dışı bırak
        outputs = model(image_tensor)  # Modelden tahmin al
        _, predicted = torch.max(outputs, 1)  # En yüksek olasılığa sahip sınıfı seç

    # Tahmin edilen sınıfı döndür
    predicted_class = class_names[predicted.item()]  # Sınıf indeksini sınıf ismine çevir
    return image, predicted_class  # Görüntü ve tahmin edilen sınıfı döndür


# Test edilecek görüntünün yolu
image_path = "Test için kullanılacak çiçek görüntüsünün tam yolu"  # Test için kullanılacak çiçek görüntüsünün tam yolu

# Görüntüyü tahmin et ve sonucu yazdır
image, predicted_class = predict_image(model_resnet, image_path, class_names)  # ResNet modeli ile tahmin yap

# Çıktıyı göster
plt.imshow(image)  # Görüntüyü matplotlib ile göster
plt.axis("off")  # Eksenleri gizle
plt.title(f"Predicted: {predicted_class}")  # Tahmin edilen sınıfı başlık olarak ekle
plt.show()  # Görüntüyü ekranda göster